In [ ]:
# Example - https://github.com/cornelliusyudhawijaya/Churn_Causality_Analysis/blob/main/Causal%20Analysis%20Do%20Why.ipynb
    
import numpy as np
import pandas as pd

from dowhy import CausalModel
import dowhy.datasets 

# Avoid printing dataconversion warnings from sklearn and numpy
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../../../data/GermanData.csv')

## Drop categorical features
data = data.drop(['1','2','4','5','8','10','11','12','14','15','16','17','18','19','20'],axis=1)


## Drop NULL values
data = data.dropna()

## for dowhy
## Change symbolics to numerics
data['sex'] = np.where(data['sex'] == 'A91', 1, data['sex'])
data['sex'] = np.where(data['sex'] == 'A92', 0, data['sex'])
data['sex'] = np.where(data['sex'] == 'A93', 1, data['sex'])
data['sex'] = np.where(data['sex'] == 'A94', 1, data['sex'])
data['sex'] = np.where(data['sex'] == 'A95', 0, data['sex'])

# mean = data.loc[:,"age"].mean()
# data['age'] = np.where(data['age'] >= mean, 1, 0)
data['age'] = np.where(data['age'] >= 25, 1, 0)
data['credit_history'] = np.where(data['credit_history'] == 'A30', 1, data['credit_history'])
data['credit_history'] = np.where(data['credit_history'] == 'A31', 1, data['credit_history'])
data['credit_history'] = np.where(data['credit_history'] == 'A32', 1, data['credit_history'])
data['credit_history'] = np.where(data['credit_history'] == 'A33', 2, data['credit_history'])
data['credit_history'] = np.where(data['credit_history'] == 'A34', 3, data['credit_history'])

data['savings'] = np.where(data['savings'] == 'A61', 1, data['savings'])
data['savings'] = np.where(data['savings'] == 'A62', 1, data['savings'])
data['savings'] = np.where(data['savings'] == 'A63', 2, data['savings'])
data['savings'] = np.where(data['savings'] == 'A64', 2, data['savings'])
data['savings'] = np.where(data['savings'] == 'A65', 3, data['savings'])

data['employment'] = np.where(data['employment'] == 'A72', 1, data['employment'])
data['employment'] = np.where(data['employment'] == 'A73', 1, data['employment'])
data['employment'] = np.where(data['employment'] == 'A74', 2, data['employment'])
data['employment'] = np.where(data['employment'] == 'A75', 2, data['employment'])
data['employment'] = np.where(data['employment'] == 'A71', 3, data['employment'])

data['Probability'] = np.where(data['Probability'] == 2, 0, 1)

non_numeric_columns = list(data.select_dtypes(exclude=[np.number]).columns)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    data[col] = le.fit_transform(data[col])

print(non_numeric_columns)

from sklearn.preprocessing import MinMaxScaler,StandardScaler

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)

print(data.head(5))

In [ ]:
# This dataset size is very small. So repeating it to make it bigger

data = pd.concat([data]*5, ignore_index=True)

In [ ]:
causal_graph = """
digraph {
credit_history;
savings;
employment;
age;
sex;
Probability;
U[label="Unobserved Confounders"];
sex -> Probability;
age -> employment; employment -> Probability;
credit_history -> Probability;
savings -> Probability;
credit_history -> savings;
U->sex;U->age;U->Probability;
}
"""

## dowhy works for binary inputs only


data['Probability'] = np.where(data['Probability'] == 0, False, True)
data['sex'] = np.where(data['sex'] == 1, True, False)

In [ ]:
# With graph
model=CausalModel(
        data = data,
        treatment="sex",
        outcome="Probability",
        graph=causal_graph.replace("\n", " ")
        )
model.view_model()


In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

In [ ]:
# Causal effect on the control group (ATC)
causal_estimate_att = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification",
        target_units = "atc")
print(causal_estimate_att)
print("Causal Estimate is " + str(causal_estimate_att.value))

# Add Random Common Cause

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "random_common_cause")
print(refutel)

# Replace Treatment with Placebo

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate,method_name="placebo_treatment_refuter",placebo_type="permute")
print(refutel)

# Remove Random Subset of Data

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "data_subset_refuter")
print(refutel)